# Worksheet 08

Name:  
UID: 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [37]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.820717111814642, 5.331423021787932, 4.577279659235284, 4.3529629169408235, 4.414164694555639, 2.539132682891533, 6.214825842266128, 6.224674288938907, 3.7737017702912485, 4.5537689417083875]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [38]:
s2 = np.random.normal(8 ,1 ,10 ).tolist()
print(s2)

[6.958815398226451, 9.359258078773456, 8.119931001733296, 8.026488537321535, 7.693393106080702, 8.11878550373014, 7.269658922388118, 8.184034757631307, 6.8930983222722695, 6.862457925704376]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [39]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop(0)
        data.append(p1)
    else:
        p2 = s2.pop(0)
        data.append(p2)
print(data)

[6.958815398226451, 4.820717111814642, 9.359258078773456, 8.119931001733296, 8.026488537321535, 7.693393106080702, 5.331423021787932, 8.11878550373014, 4.577279659235284, 4.3529629169408235]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

* Number of Components: The GMM was created with 2 mixture components.

* Mixture Weights: The weights are both 0.5 since we used a fair coin flip.

* Means: We generated s1 with a mean of 5 and s2 with a mean of 8, so the means are 5 and 8

* Covariances: We used variance 1 for both distributions, so the covariances are both 1.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [40]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2))]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.820717111814642, 5.331423021787932, 4.577279659235284, 4.3529629169408235]
[6.958815398226451, 9.359258078773456, 8.119931001733296, 8.026488537321535, 7.693393106080702, 8.11878550373014]
P(S_1) = 0.4,  P(S_2) = 0.6
mean_1 = 4.770595677444671,  mean_2 = 8.046111937644262
var_1 = 0.13220691847220362,  var_2 = 0.5070154843338079


/Users/themistoklis/opt/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Means are quite close to true values, while variances are not that close. This is probably due to the limited points that we generated.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [41]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]**0.5))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  6.958815398226451
probability of observing that point if it came from cluster 0 =  1.4982562634702234e-08
probability of observing that point if it came from cluster 1 =  0.17461227300301607
point =  4.820717111814642
probability of observing that point if it came from cluster 0 =  1.0868179546431347
probability of observing that point if it came from cluster 1 =  1.9627938266500102e-05
point =  9.359258078773456
probability of observing that point if it came from cluster 0 =  2.860961355198429e-35
probability of observing that point if it came from cluster 1 =  0.1023021196105721
point =  8.119931001733296
probability of observing that point if it came from cluster 0 =  4.1196766082368115e-19
probability of observing that point if it came from cluster 1 =  0.5572699426882103
point =  8.026488537321535
probability of observing that point if it came from cluster 0 =  4.252225964310153e-18
probability of observing that point if it came from cluster 1 =  0.5600599679238666
point 

The estimated probabilities of the data points are consistently approaching zero or one, indicating a high level of accuracy in their approximation to the true values.

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [42]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [ sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x) , sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.4,  P(S_2) = 0.6
mean_1 = 4.7703499960822215,  mean_2 = 8.045299580906725
var_1 = 0.13212798926136432,  var_2 = 0.5090804793434072


Just like before, means are really close to true values while variances not that close. However, all values moved closer to their real ones.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [43]:
prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]**0.5))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)
    
probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  6.958815398226451
probability of observing that point if it came from cluster 0 =  1.4765579180971969e-08
probability of observing that point if it came from cluster 1 =  0.1753877297437924
point =  4.820717111814642
probability of observing that point if it came from cluster 0 =  1.0870347907911688
probability of observing that point if it came from cluster 1 =  2.052580021799672e-05
point =  9.359258078773456
probability of observing that point if it came from cluster 0 =  2.705682156656321e-35
probability of observing that point if it came from cluster 1 =  0.10258582619856085
point =  8.119931001733296
probability of observing that point if it came from cluster 0 =  3.9928336418803974e-19
probability of observing that point if it came from cluster 1 =  0.5560848149290009
point =  8.026488537321535
probability of observing that point if it came from cluster 0 =  4.127769794237376e-18
probability of observing that point if it came from cluster 1 =  0.558940922480635
point = 

The probabilities where already close to 1 (and zero respectively) so we do not observe great differences or anything unexpected.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [44]:
hard_assign = []

for i in range(len(data)):
    if prob_s0_x[i] > prob_s1_x[i]:
        hard_assign.append(0)
    else:
        hard_assign.append(1)
print("Hard Assignments:", hard_assign)

Hard Assignments: [1, 0, 1, 1, 1, 1, 0, 1, 0, 0]


We also display the dataset, to provide an additional layer of evaluation to complement the already high probabilities.

In [45]:
print(data)

[6.958815398226451, 4.820717111814642, 9.359258078773456, 8.119931001733296, 8.026488537321535, 7.693393106080702, 5.331423021787932, 8.11878550373014, 4.577279659235284, 4.3529629169408235]
